# Evaluation of Cross Validation

### Step 1 Load Libraries

In [1]:
import sys
sys.path.append('../python/')
from tqdm import tqdm
from glob import glob
import pandas as pd
from parametrized_bootstrapping_model import ParametrizedBootstrappingModel, ReturnAlways1Model, ReturnAlways0Model
from result_analysis_utils import load_ground_truth_data, load_evaluations, run_cross_validation, load_cross_validation_results
SEARCH_SPACE= [0, 1, 2] + list(range(5,96, 5)) + [98, 99, 100]

TREC = 'trec13'
RUNS=110
BOOTSTRAPPING_TYPE = 'bs-p-1000-ndcg@10-ndcg@10'
#BOOTSTRAPPING_TYPE = 'bs-run-and-pool-dependent-1000-ndcg@10-ndcg@10'
#BOOTSTRAPPING_TYPE = 'bs-pool-dependent-1000-ndcg@10-ndcg@10'
#BOOTSTRAPPING_TYPE = 'bs-run-dependent-1000-ndcg@10-ndcg@10'

### Step 2 Load Dataset

In [2]:
if 'eval_df' not in locals():
    eval_df = load_evaluations(tqdm(glob('../resources/eval/trec-system-runs/' + TREC + '/*.jsonl')))

eval_df.head(2)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 877/877 [01:35<00:00,  9.19it/s]


,,"(depth-10-complete, unjudged@10)","(depth-10-incomplete, unjudged@10)","(depth-20-complete, unjudged@10)","(depth-20-incomplete, unjudged@10)","(depth-10-incomplete, bs-p-1000-ndcg@10-ndcg@10)","(depth-10-incomplete, bs-run-and-pool-dependent-1000-ndcg@10-ndcg@10)","(depth-10-complete, condensed-ndcg@10)","(depth-10-incomplete, condensed-ndcg@10)","(depth-20-complete, condensed-ndcg@10)","(depth-20-incomplete, condensed-ndcg@10)",...,"(depth-10-incomplete, bs-run-dependent-1000-ndcg@10-ndcg@10)","(depth-10-complete, residual-ndcg@10-min)","(depth-10-complete, residual-ndcg@10-max)","(depth-10-incomplete, residual-ndcg@10-min)","(depth-10-incomplete, residual-ndcg@10-max)","(depth-20-complete, residual-ndcg@10-min)","(depth-20-complete, residual-ndcg@10-max)","(depth-20-incomplete, residual-ndcg@10-min)","(depth-20-incomplete, residual-ndcg@10-max)",run
run,,,,,,,,,,,,,,,,,,,,,,
JuruDes,0,"{""301"": 0.0, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.2, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.0, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.1, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": [0.0, 0.07336392209936005, 0.293455688...","{""301"": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0.12987501165334076, ""302"": 0.77215962...","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0.22009176629808017, ""302"": 0.77215962...",...,"{""301"": [0.07336392209936005, 0.07336392209936...","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0, ""302"": 0.7721596254947202, ""303"": 0...","{""301"": 0.2934556883974402, ""302"": 0.772159625...","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0.2934556883974402, ""302"": 0.772159625...",JuruDes
JuruDesAggr,0,"{""301"": 0.0, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.2, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.0, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.1, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": [0.2180164145324244, 0.236905308719656...","{""301"": [0.15176219107803537, 0.15176219107803...","{""301"": 0.3557772116892465, ""302"": 0.772159625...","{""301"": 0.2180164145324244, ""302"": 0.772159625...","{""301"": 0.3557772116892465, ""302"": 0.772159625...","{""301"": 0.4265076985331692, ""302"": 0.772159625...",...,"{""301"": [0.15176219107803537, 0.21801641453242...","{""301"": 0.23690530871965634, ""302"": 0.77215962...","{""301"": 0.23690530871965634, ""302"": 0.77215962...","{""301"": 0.15176219107803537, ""302"": 0.77215962...","{""301"": 0.3031595321740454, ""302"": 0.772159625...","{""301"": 0.23690530871965634, ""302"": 0.77215962...","{""301"": 0.23690530871965634, ""302"": 0.77215962...","{""301"": 0.23690530871965634, ""302"": 0.77215962...","{""301"": 0.3031595321740454, ""302"": 0.772159625...",JuruDesAggr


In [3]:
load_ground_truth_data(
            df=eval_df[eval_df['run'] == eval_df.iloc[0]['run']],
            ground_truth_measure='ndcg',
            depth=10,
            input_measure=BOOTSTRAPPING_TYPE,
            random_state=3
        )

,run,query,x,y,measures,split
0,JuruDes,301,"[0.0, 0.07336392209936005, 0.2934556883974402,...",0.220092,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",0
1,JuruDes,302,"[0.7721596254947203, 0.7721596254947203, 0.772...",0.772160,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",0
2,JuruDes,303,"[0.0874717141014082, 0.0874717141014082, 0.087...",0.087472,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",0
3,JuruDes,304,"[0.33480714634441017, 0.33480714634441017, 0.3...",0.334807,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",1
4,JuruDes,305,"[0.0, 0.0, 0.0, 0.11284514134893527, 0.0, 0.0,...",0.000000,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",1
...,...,...,...,...,...,...
244,JuruDes,696,"[0.6563033086078546, 0.6563033086078546, 0.656...",0.656303,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",1
245,JuruDes,697,"[0.4590828233913731, 0.4590828233913731, 0.459...",0.459083,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",0
246,JuruDes,698,"[0.4932032819614565, 0.3390458811024219, 0.455...",0.291736,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",1
247,JuruDes,699,"[0.6410457898283652, 0.6410457898283652, 0.641...",0.641046,"{'x': 'bs-p-1000-ndcg@10-ndcg@10', 'y': 'ndcg@...",0


### Step 3: Do the actual cross validation

In [4]:
def run_cross_validation_on_runs(runs):
    ret = []
    for i in tqdm(list(range(runs))):
        ground_truth_data = load_ground_truth_data(
            df=eval_df[eval_df['run'] == eval_df.iloc[i]['run']],
            ground_truth_measure='ndcg',
            depth=10,
            input_measure=BOOTSTRAPPING_TYPE,
            random_state=3
        )
        
        models = [ParametrizedBootstrappingModel(i, SEARCH_SPACE) for i in ['rmse[0.6,1]', 'rmse[0.8,1]', 'rmse', 'rmse[1,2]', 'rmse[1,3]', 'rmse[0.1,5]', 'rmse[0.1,10]', 'rmse[0.1,100]', 'rmse[0,1]']] + [ReturnAlways1Model(), ReturnAlways0Model()]

        for model in models:
            tmp = run_cross_validation(ground_truth_data, model)
            
            ret += [tmp]
            
            tmp = tmp.to_json(lines=True, orient='records')
            with open('cross-validation-results/' TREC + '/' + BOOTSTRAPPING_TYPE + '-results.jsonl', 'a+') as f:
                f.write(tmp)

    return pd.concat(ret)

!rm -f cross-validation-results/{TREC}/{BOOTSTRAPPING_TYPE}-results.jsonl
results = run_cross_validation_on_runs(RUNS)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 95/95 [42:33<00:00, 26.88s/it]
